In [1]:
import numpy as np
import pandas as pd
from google.colab import files
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


###Side task: split training set into training and validation

In [4]:
# open file and read the content in a list
train_notes_cleaned=[]
with open('gdrive/MyDrive/BMI707_Final_Project/train_notes_cleaned.txt', 'r') as f1:
    for line in f1:
        # remove linebreak which is the last character of the string
        note = line[:-1]

        # add item to the list
        train_notes_cleaned.append(note)        

#read in continuous outcomes
train_data_balanced = pd.read_csv("gdrive/MyDrive/BMI707_Final_Project/train_data_balanced.csv")
train_labels_ordinal = np.array(train_data_balanced.outcome_count) 
train_labels_binary = np.array(train_data_balanced.binary_outcome)      

In [9]:
#attach binary outcome, outcome count, and ID to padded_notes_trainNEW.csv

padded_notes_trainNEW = pd.read_csv("gdrive/MyDrive/BMI707_Final_Project/padded_notes_trainNEW.csv")
padded_notes_trainNEW["outcome_count"] = train_data_balanced.outcome_count
padded_notes_trainNEW["ID"] = train_data_balanced.ID



In [22]:
#attach binary outcome, outcome count, and ID to train_notes_cleaned
type(train_notes_cleaned)
train_notes_cleaned_df = pd.DataFrame(train_notes_cleaned)
train_notes_cleaned_df["outcome_count"] = train_data_balanced.outcome_count
train_notes_cleaned_df["binary_count"] = train_data_balanced.binary_outcome
train_notes_cleaned_df["ID"] = train_data_balanced.ID

In [23]:
import random
new_data = train_data_balanced
#split ids by binary outcome, since test and train should be balanced
new_data_outcome0 = new_data[new_data.binary_outcome == 0]
new_data_outcome1 = new_data[new_data.binary_outcome == 1]

#get unique ids for each outcome
patient_ids_outcome0 = new_data_outcome0.ID.unique()
print(len(patient_ids_outcome0))
patient_ids_outcome1 = new_data_outcome1.ID.unique()
print(len(patient_ids_outcome1))


#shuffle each id list
random.Random(4).shuffle(patient_ids_outcome0)
random.Random(4).shuffle(patient_ids_outcome1)

#split ids  in EACH list
total_ids = len(patient_ids_outcome0) #get total number of unique ids
split = round(total_ids*.8) #find the 80% split index for training
train_split_ids_outcome0 = patient_ids_outcome0[:split]
val_split_ids_outcome0 = patient_ids_outcome0[split:]

total_ids = len(patient_ids_outcome1) #get total number of unique ids
split = round(total_ids*.8) #find the 80% split index for training
train_split_ids_outcome1 = patient_ids_outcome1[:split]
val_split_ids_outcome1 = patient_ids_outcome1[split:]

#recombine balanced patients  to make full train and test
train_split_ids = np.concatenate((train_split_ids_outcome0, train_split_ids_outcome1), axis=0)
val_split_ids = np.concatenate((val_split_ids_outcome0, val_split_ids_outcome1), axis=0)




9943
4586


In [24]:
#split full dataset, train_notes_cleaned_df,  and padded_notes_trainNEW based on ids
train_data_full_dataset = new_data.loc[new_data['ID'].isin(train_split_ids)]
val_data_full_dataset = new_data.loc[new_data['ID'].isin(val_split_ids)]

train_data_notes_cleaned = train_notes_cleaned_df.loc[train_notes_cleaned_df['ID'].isin(train_split_ids)]
val_data_notes_cleaned = train_notes_cleaned_df.loc[train_notes_cleaned_df['ID'].isin(val_split_ids)]

train_data_padded = padded_notes_trainNEW.loc[padded_notes_trainNEW['ID'].isin(train_split_ids)]
val_data_padded = padded_notes_trainNEW.loc[padded_notes_trainNEW['ID'].isin(val_split_ids)]



In [33]:
print(val_data_padded.shape)
print(train_data_notes_cleaned.shape)
train_data_full_dataset.shape

(26056, 516)
(105635, 4)


(105635, 12)

In [34]:
#write data sets to cv
train_data_full_dataset.to_csv('gdrive/MyDrive/BMI707_Final_Project/TRAINING_full_dataset.csv',index=False)
val_data_full_dataset.to_csv('gdrive/MyDrive/BMI707_Final_Project/VALIDATION_full_dataset.csv',index=False)

train_data_notes_cleaned.to_csv('gdrive/MyDrive/BMI707_Final_Project/TRAINING_notes_cleaned.csv',index=False)
val_data_notes_cleaned.to_csv('gdrive/MyDrive/BMI707_Final_Project/VALDIATION_notes_cleaned.csv',index=False)

train_data_padded.to_csv('gdrive/MyDrive/BMI707_Final_Project/TRAINING_notes_padded.csv',index=False)
val_data_padded.to_csv('gdrive/MyDrive/BMI707_Final_Project/VALDIATION_notes_padded.csv',index=False)


###End of side task

In [ ]:
#read in data
uploaded = files.upload()

Saving joined_notes_outcome_patient_data.csv to joined_notes_outcome_patient_data (1).csv


In [ ]:
import io
data = pd.read_csv(io.BytesIO(uploaded['joined_notes_outcome_patient_data.csv']))
data.head()

In [ ]:
#select relevant columns
notes = data[["TEXT"]]
binary_labels = data[["binary_outcome"]]

Split into test and train datasets. We need to do this before becaue the Tokenizer should ONLY be generated on the training data


In [ ]:
#create a new ID system so dont have to continue using two columns

#merge the 2 ids in new column
data["ID"] = data.SUBJECT_ID.astype(str) + "-" + data.HADM_ID.astype(str)
data.head()

#remove old ids
new_data = data.drop(["SUBJECT_ID", "HADM_ID"], axis=1)



In [ ]:
import random

#split ids by binary outcome, since test and train should be balanced
new_data_outcome0 = new_data[new_data.binary_outcome == 0]
new_data_outcome1 = new_data[new_data.binary_outcome == 1]

#get unique ids for each outcome
patient_ids_outcome0 = new_data_outcome0.ID.unique()
print(len(patient_ids_outcome0))
patient_ids_outcome1 = new_data_outcome1.ID.unique()
print(len(patient_ids_outcome1))


#shuffle each id list
random.Random(4).shuffle(patient_ids_outcome0)
random.Random(4).shuffle(patient_ids_outcome1)

#split ids  in EACH list
total_ids = len(patient_ids_outcome0) #get total number of unique ids
split = round(total_ids*.8) #find the 80% split index for training
train_ids_outcome0 = patient_ids_outcome0[:split]
test_ids_outcome0 = patient_ids_outcome0[split:]

total_ids = len(patient_ids_outcome1) #get total number of unique ids
split = round(total_ids*.8) #find the 80% split index for training
train_ids_outcome1 = patient_ids_outcome1[:split]
test_ids_outcome1 = patient_ids_outcome1[split:]

#recombine balanced patients  to make full train and test
train_ids = np.concatenate((train_ids_outcome0, train_ids_outcome1), axis=0)
test_ids = np.concatenate((test_ids_outcome0, test_ids_outcome1), axis=0)




In [ ]:
#check lists
print(len(train_ids))
len(test_ids)

In [ ]:
#split full dataset based on ids
train_data = new_data.loc[new_data['ID'].isin(train_ids)]
test_data = new_data.loc[new_data['ID'].isin(test_ids)]

#split into input (notes) and output (labels)
train_notes = train_data.TEXT
train_labels = train_data.binary_outcome
test_notes = test_data.TEXT
test_labels = test_data.binary_outcome

In [ ]:
#check number of notes for train and test
print(str(len(train_notes)) + ' notes in training data which has 104 patients') # how did we know it is 104?
print(str(len(test_notes)) + ' notes in test data which has 27 patients')

##Download datasets

In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

#write data sets to cv
train_data.to_csv('/content/gdrive/My Drive/train_data_full.csv',index=False)
test_data.to_csv('/content/gdrive/My Drive/test_data_full.csv',index=False)

files.download('/content/gdrive/My Drive/train_data_full.csv')
files.download('/content/gdrive/My Drive/test_data_full.csv')

##Reload train/test datasets here (can run full notebook from here)


In [ ]:
#RELOAD DATASSETS

# from google.colab import files
# from google.colab import drive
# drive.mount('/content/gdrive')

# !ls

# train_data = pd.read_csv('/content/gdrive/My Drive/bmi707/train_data_balanced.csv')
# test_data = pd.read_csv('/content/gdrive/My Drive/bmi707/test_data_balanced.csv')

In [ ]:
#split into input (notes) and output (labels)
train_notes = train_data.TEXT
train_labels = train_data.binary_outcome
test_notes = test_data.TEXT
test_labels = test_data.binary_outcome

#check number of notes for train and test
print(str(len(train_notes)) + ' notes in training data which has 104 patients')
print(str(len(test_notes)) + ' notes in test data which has 27 patients')

##Sources: 

https://medium.com/@nwamaka_41565/predicting-hospital-readmission-using-nlp-5f0fe6f1a705

##ClinicalBert requires minimal preprocessing: (excerpt from above article)

First, words are converted to lowercase

Line breaks are removed

Carriage returns are removed

De-identified the personally identifiable info inside the brackets

Remove special characters like ==, −−

The SpaCy sentence segmentation package is used to segment each note . i think this is implemented automatically with NLTK.

##Other Notes: ((excerpt from above article)

Since clinical notes don’t follow rigid standard language grammar, we find rule-based segmentation has better results than dependency parsing-based segmentation. Various segmentation signs that misguide rule-based segmentators are removed or replaced.

For example 1.2 would be removed.

M.D., dr. would be replaced with with MD, Dr

Clinical notes can include various lab results and medications that also contain numerous rule-based separators, such as 20mg, p.o., q.d.. (where q.d. means one a day and q.o. means to take by mouth. To address this, segmentations that have less than 20 words are fused into the previous segmentation so that they are not singled out as different sentences.


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re

from nltk import pos_tag
from nltk.stem import WordNetLemmatizer # lemmatizes word based on its parts of speech


Look at stop words and punctuation that will be removed

In [ ]:
nltk.download('stopwords')
print('Punctuation:', string.punctuation)
print('NLTK English Stop Words:', '\n', stopwords.words('english'))

Convert POS tags from string package to what is used in WordNet (NLTK) package. all from the article

In [ ]:
def convert_tag(treebank_tag):
    '''Convert Treebank tags to WordNet tags'''
    if treebank_tag.startswith('J'):
        return 'a'
    elif treebank_tag.startswith('V'):
        return 'v'
    elif treebank_tag.startswith('N'):
        return 'n'
    elif treebank_tag.startswith('R'):
        return 'r'
    else:
        return 'n' # if no match, default to noun

Define lemmatizer (stemming) and preprocess_and_tokenize. these were all from the article! 

In [ ]:
def lemmatizer(tokens):
    '''
    Performs lemmatization.
    Params:
        tokens (list of strings): cleaned tokens with stopwords removed
    Returns:
        lemma_words (list of strings): lemmatized words
    '''  
    # POS-tag your data before lemmatizing
    tagged_words = pos_tag(tokens) # outputs list of tuples [('recent', 'JJ'),...]
    
    
    # Lemmatize using WordNet's built-in morphy function. Returns the input word unchanged if it cannot be found in WordNet.
    wnl = WordNetLemmatizer()
    
    lemma_words = []
    
    # Lemmatize list of tuples, output a list of strings
    for tupl in tagged_words:
        lemma_words.append(wnl.lemmatize(tupl[0], convert_tag(tupl[1])))
    
    return lemma_words
def preprocess_and_tokenize(text):
    '''
    Clean the data.
    Params:
        text (string): full original, uncleaned text
    Returns:
        lemmatized_tokens (list of strings): cleaned words
    '''
    # Make text lowercase
    text = text.lower()
    
    # Remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) 
    
    # Remove numbers and words that contain numbers
    text = re.sub('\w*\d\w*', '', text)
    
    # Remove newline chars and carriage returns
    text = re.sub('\n', '', text)
    text = re.sub('\r', '', text)
    
    # Tokenize
    word_tokens = word_tokenize(text) 
    
    # Remove stop words
    tokens = [word for word in word_tokens if word not in stopwords.words('english')]
# Call lemmatizer function above to perform lemmatization
    lemmatized_tokens = lemmatizer(tokens)
    
    return lemmatized_tokens

In [ ]:
#manually run preprocess_and_tokenize on every note BOTH train and test data

#additional nltk packages needed 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')



cleaned_train_notes = [preprocess_and_tokenize(x) for x in train_notes]  
cleaned_test_notes = [preprocess_and_tokenize(x) for x in test_notes]  

#less efficient ways of ^^
# cleaned_train_notes = []
# for i in train_notes:
#   note = preprocess_and_tokenize(i)
#   cleaned_train_notes.append(note)
# cleaned_test_notes = []
# for i in test_notes:
#   note = preprocess_and_tokenize(i)
#   cleaned_test_notes.append(note)

##Download cleaned train and test notes

In [ ]:
# write cleaned tokenized notes to text file
with open('/content/gdrive/My Drive/train_notes_cleaned_full.txt', 'w') as f:
  for item in cleaned_train_notes:
        f.write("%s\n" % item)
with open('/content/gdrive/My Drive/test_notes_cleaned_full.txt', 'w') as f:
  for item in cleaned_test_notes:
        f.write("%s\n" % item)

files.download('/content/gdrive/My Drive/train_notes_cleaned_full.txt')
files.download('/content/gdrive/My Drive/test_notes_cleaned_full.txt')


In [ ]:
print(cleaned_test_notes[1:3])

##Convert each note from word sequence to integer vector

This is equivalent to text_to_sequences function in BMI 707 code. I could not do that bc I had to make a customized tokenize function and could not use the default keras Tokenizer class.


In [ ]:
#rejoin tokenized words from preprocess_and_tokenize back into strings
separator=' '
rebuilt_train_notes = [separator.join(i) for i in cleaned_train_notes]   
rebuilt_test_notes = [separator.join(i) for i in cleaned_test_notes]    

from tensorflow.keras.preprocessing.text import Tokenizer

#create keras tokenizer
max_words = 10000
tokenizer = Tokenizer(max_words, oov_token='OOV')
tokenizer.fit_on_texts(rebuilt_train_notes)
vocab_size = max_words + 1

#convert text to sequences
cleaned_train_notesNEW = tokenizer.texts_to_sequences(rebuilt_train_notes)
cleaned_test_notesNEW = tokenizer.texts_to_sequences(rebuilt_test_notes)



In [ ]:
new_train_seqs = cleaned_train_notesNEW
new_test_seqs = cleaned_test_notesNEW

seq_len = 512
padded_notes_trainNEW = pad_sequences(new_train_seqs, maxlen=seq_len)
padded_notes_testNEW = pad_sequences(new_test_seqs, maxlen=seq_len)
print(padded_notes_trainNEW.shape)


#write NEW padded notes to text file
with open('/content/gdrive/My Drive/padded_notes_trainNEW_full.txt', 'w') as f:
  for item in padded_notes_trainNEW:
        f.write("%s\n" % item)
with open('/content/gdrive/My Drive/padded_notes_testNEW_full.txt', 'w') as f:
  for item in padded_notes_testNEW:
        f.write("%s\n" % item)


#Do not pay attention to anything below here


In [ ]:
#convert each note from word sequence to integer vector
import itertools
from gensim.corpora.dictionary import Dictionary

#create dictionary for training data notes only
flat_notes = list(itertools.chain(*cleaned_train_notes))
note_vocab = Dictionary([flat_notes])

#get size of vocabulary - should be less than 10,000
vocab_size = len(np.unique(np.array(flat_notes)))
oov_int = vocab_size + 5 #make token for out of vocab

#remake integer sequences from tokenized notes for train data
train_sequences = []
for i in cleaned_train_notes:
  
  seq = [note_vocab.token2id[token] for token in i]
  train_sequences.append(seq)

#remake integer sequences from tokenized notes for test data 
test_sequences = []
for i in cleaned_test_notes:
  
    new_seq = [note_vocab.token2id[token] if token in note_vocab else oov_int for token in i ] 

  # for j in i:      very slow way
  #   new_seq=[]
  #   if j in note_vocab:
  #     new_seq.append(note_vocab.token2id[j])
  #   else:   new_seq.append(oov_int) #append oov index

    test_sequences.append(new_seq)  #append full sequence to test_sequences

##I think we can use the gensim Dictionary called node_vocab to remap integers back into words after output is generated.

In [ ]:
#get size of vocabulary - should be less than 10,000
len(np.unique(np.array(flat_notes)))

## Pad sequences and trunate long sequences

Next we 0-pad the sequences, and short sequences will have 0s prepended so that each sequence is exactly 100 integers long. Note long sentences will be trimmed so that only the first 100 words are used.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
seq_len = 512
padded_notes_train = pad_sequences(train_sequences, maxlen=seq_len)
padded_notes_test = pad_sequences(test_sequences, maxlen=seq_len)
print(padded_notes_train.shape)


##Save test and train datasets

In [ ]:
# leaned_test_notes = [preprocess_and_tokenize(x) for x in test_notes]  

#write padded notes to text file
with open('/content/gdrive/My Drive/train_note_seqs.txt', 'w') as f:
  for item in padded_notes_train:
        f.write("%s\n" % item)
with open('/content/gdrive/My Drive/test_note_seqs.txt', 'w') as f:
  for item in padded_notes_test:
        f.write("%s\n" % item)

files.download('/content/gdrive/My Drive/train_note_seqs.txt')
files.download('/content/gdrive/My Drive/test_note_seqs.txt')  

# #write original 
# with open('/content/gdrive/My Drive/train_note_seqs.txt', 'w') as f:
#   f.write(padded_notes_train)
# with open('/content/gdrive/My Drive/test_note_seqs.txt', 'w') as f:
#   f.write(padded_notes_test)



# files.download('/content/gdrive/My Drive/train_data_balanced.csv')
# files.download('test_data_balanced.csv')

In [ ]:
#Use this code to count the nubmer of unique words (ie True vocab size). We do not use this for anything else for now, might be used for Word2Vec model

#first build CountVectorizer object (counts the number of words )
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(max_features=10000, #set to 10,000 words
                       tokenizer=preprocess_and_tokenize)

vect.fit(train_notes.values.astype(str))

#additional nltk packages needed 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

#####This is the only part of this that matters

#check how many words are in vectorizer (we set max as 10,000)
vocab_size = len(vect.vocabulary_)
print(vocab_size)

# # build tokenizer
# tokenizer = vect.build_tokenizer()

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
texts = train_data['Text']
test_sentences = test_data['Text']
labels = train_data['Label']

vocab_size = max_words + 1

sequences = token.texts_to_sequences(texts)
test_sequences = token.texts_to_sequences(test_sentences)



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(max_features=10000, #set to 10,000 words
                       tokenizer=preprocess_and_tokenize)

#additional nltk packages needed 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# I applied .astype(str) to fix the ValueError: np.nan is an invalid document, expected byte or unicode string.xc
# create the vectorizer
vect.fit(train_notes.values.astype(str))

In [ ]:
#check how many words are in vectorizer (we set max as 10,000)
print(len(vect.vocabulary_))

#check number of train notes
train_notes.shape

In [ ]:
#Actually create a vector by passing the text into the vectorizer to get back counts.

note_matrix = vect.transform(train_notes.astype(str))
print(note_matrix.shape) #this should be number_of_notes X vocab_size

In [ ]:
note_matrix